In [27]:
from json import load
from pathlib import Path
from flightanalysis import ScheduleInfo
import numpy as np
import pandas as pd

mandetails = ScheduleInfo('f3a', 'p25').manoeuvre_details()

results = dict()
for path in Path('./output/').glob('*.json'):
    data = load(path.open())

    mscores = {man['name']: list(man['history'].values())[-1]['results'][4]['score']['total'] for man in data['mans']}
    total = sum([ms*md.k for ms, md in zip(mscores.values(), mandetails)])
    results[path.stem] = dict(
        lat=data['origin']['lat'],
        lng=data['origin']['lng'],
        alt=data['origin']['alt'],
        total = total,
        **mscores
    )

df = pd.DataFrame(results).T.sort_values('total', ascending=False)

df.insert(3, 'rank', np.arange(len(df)) + 1)
pd.options.display.float_format = '{:,.2f}'.format
df.iloc[:,3:]

,rank,total,trgle,hSqL,sqL,fig9,rollC,stall,dImm,hB,loop,hSqL2,hClov,rEt,iSpin,tHat,Z,Com,figS
Halo_UK,1,460.04,6.96,9.46,5.44,6.76,8.07,8.89,7.74,7.66,7.20,8.51,7.25,8.15,8.15,8.03,8.74,6.82,6.81
Mach44_UK,2,392.56,4.13,9.00,5.11,5.55,8.43,8.73,6.52,4.93,6.09,8.58,5.01,6.86,7.84,7.98,7.02,5.78,5.23
Ddo_JPN,3,380.89,5.74,8.75,5.78,8.64,8.36,8.80,5.57,7.43,1.36,8.58,2.98,7.16,8.39,8.18,6.61,6.63,4.69
Jun_JPN,4,379.19,6.09,8.85,5.51,7.88,8.21,7.36,7.33,6.56,2.07,7.57,3.87,7.18,7.83,7.52,8.06,6.85,3.15
Wang_TWN,5,326.94,3.84,7.93,2.56,7.51,7.87,6.18,1.89,6.40,4.37,8.00,3.21,7.04,3.59,2.85,8.04,6.86,6.66
Joy_TWN,6,310.98,0.00,8.55,4.52,5.46,7.06,7.08,5.18,5.86,1.99,6.93,5.80,6.43,6.17,6.01,5.28,6.05,2.56
Kameo_JPN,7,289.83,4.68,9.12,2.79,8.23,7.97,5.69,1.36,6.14,1.73,6.46,1.91,3.23,5.25,6.71,5.11,5.43,5.89


In [28]:
import plotly.express as px

px.scatter_geo(lat=df.lat, lon=df.lng, color=df.total, text=df.index, projection='natural earth').show()